## Importing Libraries

In [1]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
import os
os. environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Loading Data

In [4]:
SEED = 2017
DATA_DIR = 'data/' 

In [5]:
files = glob.glob(DATA_DIR + "*.wav")
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

In [6]:
print('Training examples: {}'.format(len(X_train)))
print('Validation examples: {}'.format(len(X_val)))

Training examples: 1920
Validation examples: 480


In [7]:
labels = []

for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    
    if label not in labels:
        labels.append(label)

In [8]:
print(labels)

['Vicki', 'Ralph', 'Alex', 'Bruce', 'Victoria', 'Tom', 'Fred', 'Albert', 'Agnes', 'Daniel', 'Princess', 'Junior', 'Steffi', 'Kathy', 'Samantha']


## Data Preprocessing

In [9]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

LabelBinarizer()

In [10]:
def one_hot_encode(label):
    return label_binarizer.transform(label)

In [11]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [12]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

## Model Training

In [13]:
learning_rate = 0.001
batch_size = 64
n_epochs = 50
dropout = 0.5

In [14]:
input_shape = (n_features, max_length)
steps_per_epoch = 50

In [15]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

In [16]:
opt = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 256)           345088    
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                                                 
 dense (Dense)               (None, 128)               655488    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 15)                1935      
                                                                 
Total params: 1,002,511
Trainable params: 1,002,511
Non-trainable params: 0
_________________________________________________________________


In [17]:
callbacks = [ModelCheckpoint('checkpoints/voice_recognition_best_model_{epoch:02d}.hdf5', save_best_only=True),
            EarlyStopping(monitor='val_acc', patience=2)]

In [ ]:
history = model.fit_generator(
    generator=batch_generator(X_train, batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=n_epochs,
    verbose=1,
    validation_data=batch_generator(X_val, 32),
    validation_steps=5,
    callbacks=callbacks
)

/tmp/ipykernel_11706/2466081567.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50
45/50 [==========================>...] - ETA: 7s - loss: 2.3971 - accuracy: 0.2031